In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import files

In [ ]:
# !pip install read_parquet
!pip install fastparquet
import pyarrow.parquet as pq

# df.to_csv('filenamteste.csv')

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

In [ ]:
import fastparquet

In [ ]:
files.upload()

In [ ]:
train_df = pd.read_parquet('train.parquet', engine='fastparquet')

In [ ]:
train_df

,id,rko_start_months,max_end_fact_fin_deals,max_end_plan_non_fin_deals,max_start_fin_deals,max_start_non_fin_deals,min_end_fact_fin_deals,min_end_plan_non_fin_deals,min_start_fin_deals,min_start_non_fin_deals,...,cnt_days_cred_g_oper_3m,sum_deb_h_oper_3m,cnt_deb_h_oper_3m,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m,target_1,target_2,total_target
0,0,-1.279132,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.352516,-0.081310,0.210189,1.140808,0.647477,0.810887,1.909568,0,0,0
1,1,-1.142591,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.188287,-0.650616,-0.316563,-1.136536,-0.127326,-0.147319,-0.517805,0,0,0
2,2,1.812270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.323487,0.632731,2.541558,3.017482,0.033394,0.019972,0.578428,0,0,0
3,3,-0.479407,NaN,0.356677,NaN,-0.332867,NaN,-0.704164,NaN,0.905748,...,-0.323487,-0.003549,-0.219016,0.228624,-0.428185,-0.220932,-0.596108,0,0,0
4,4,-1.503690,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.323487,0.182726,-0.287299,-0.528921,-0.563335,-0.021506,-0.361201,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,299995,0.140494,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.082114,0.276906,0.346755,1.535162,0.718633,-0.242059,-0.047991,0,0,0
299996,299996,-0.578334,NaN,-0.773909,NaN,0.182200,NaN,-0.690771,NaN,-0.519096,...,-0.323487,-0.264553,-0.316563,-0.472916,-0.053175,0.222753,-0.596108,0,0,0
299997,299997,2.052711,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.323487,0.139359,-0.316563,-0.768966,-0.115407,-0.054465,-0.047991,0,0,0
299998,299998,1.051685,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.188287,-0.122459,-0.316563,-0.317586,0.829699,-0.416806,-0.596108,0,1,1


In [ ]:
# train_df.channel_code = pd.Categorical(train_df.channel_code)
# # train_df['code1'] = train_df.channel_code.cat.codes
train_df.city = pd.Categorical(train_df.city)
train_df['code2'] = train_df.city.cat.codes
train_df.city_type = pd.Categorical(train_df.city_type)
train_df['code3'] = train_df.city_type.cat.codes
# # #train_df.index_city_code = pd.Categorical(train_df.index_city_code)
# # #train_df['code4'] = train_df.index_city_code.cat.codes
train_df.ogrn_month = pd.Categorical(train_df.ogrn_month)
train_df['code5'] = train_df.ogrn_month.cat.codes
# train_df.ogrn_year = pd.Categorical(train_df.ogrn_year)
# train_df['code6'] = train_df.ogrn_year.cat.codes
train_df.branch_code = pd.Categorical(train_df.branch_code)
train_df['code7'] = train_df.branch_code.cat.codes
train_df.okved = pd.Categorical(train_df.okved)
train_df['code8'] = train_df.okved.cat.codes
train_df.segment = pd.Categorical(train_df.segment)
train_df['code9'] = train_df.segment.cat.codes

In [ ]:
a = []
for i in train_df.columns:
  if train_df.dtypes[i] == 'object':
    a.append(i)
    print(i)
train_df.drop(a, axis=1, inplace=True)

nulls = train_df.isnull().sum()/len(train_df)*100
for i in train_df.columns:
    median = train_df[i].median()
    train_df[i].fillna(median, inplace=True)

channel_code
index_city_code
ogrn_year


TypeError: ignored

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
correlation_matrix = train_df.corr().abs()

#Визуализируем матрицу корреляции с помощью тепловой карты
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('Матрица корреляции')
plt.show()

In [ ]:
s = correlation_matrix.unstack().abs()
so = s.sort_values(kind="quicksort")
print(so[(so>0.9) & (so<0.99)].index)

In [ ]:
train_df["result_of_cor_1"] = train_df.cnt_cred_d_oper_3m * train_df.cnt_cred_d_oper_1m
train_df["result_of_cor_2"] = train_df.sum_cred_e_oper_1m * train_df.sum_deb_e_oper_1m
train_df["result_of_cor_3"] = train_df.cnt_days_deb_g_oper_3m * train_df.cnt_days_deb_g_oper_1m
train_df["result_of_cor_4"] = train_df.cnt_deb_h_oper_3m * train_df.cnt_deb_h_oper_1m
train_df["result_of_cor_5"] = train_df.cnt_cred_g_oper_3m * train_df.cnt_cred_g_oper_1m# - берем только 4 штуки т.к. результат лучше
train_df["result_of_cor_6"] = train_df.cnt_cred_f_oper_3m * train_df.cnt_cred_f_oper_1m
train_df["result_of_cor_7"] = train_df.ft_registration_date * train_df.ogrn_exist_months
train_df["result_of_cor_8"] = train_df.cnt_deb_f_oper_3m * train_df.cnt_cred_f_oper_1m
train_df["result_of_cor_10"] = train_df.cnt_cred_e_oper_3m * train_df.cnt_cred_e_oper_1m
train_df["result_of_cor_11"] = train_df.cnt_days_cred_h_oper_1m * train_df.cnt_days_cred_h_oper_3m
train_df["result_of_cor_12"] = train_df.cnt_cred_f_oper_1m * train_df.cnt_deb_f_oper_3m
train_df["result_of_cor_13"] = train_df.cnt_days_cred_f_oper_1m * train_df.cnt_deb_f_oper_1m
train_df["result_of_cor_14"] = train_df.cnt_cred_h_oper_1m * train_df.cnt_cred_h_oper_3m
train_df["result_of_cor_15"] = train_df.cnt_days_deb_f_oper_1m * train_df.cnt_cred_f_oper_3m


In [ ]:
train_df


In [ ]:
for col in train_df.columns:
    pct_missing = np.mean(train_df[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

In [ ]:
#train_df.hist(figsize=(40,40))

In [ ]:
train_df.info()

In [ ]:
cat_cols = [
    'channel_code', 'city', 'city_type',
    'index_city_code', 'ogrn_month', 'ogrn_year',
    'branch_code', 'okved', 'segment'
]

In [ ]:
train_df[cat_cols] = train_df[cat_cols].astype("category")

In [ ]:
num_rows = len(train_df.index)
low_information_cols = [] #

for col in train_df.columns:
    cnts = train_df[col].value_counts(dropna=False)
    top_pct = (cnts/num_rows).iloc[0]

    if top_pct > 0.95:
        low_information_cols.append(col)
        print('{0}: {1:.5f}%'.format(col, top_pct*100))
        print(cnts)
        print()

In [ ]:
 train_df.drop(["id","target_1", "target_2"], axis=1, inplace=True)

In [ ]:
train_df.drop([
#'rko_start_months',
'max_end_fact_fin_deals',
'max_end_plan_non_fin_deals',
'max_start_fin_deals',
'max_start_non_fin_deals',
'min_end_fact_fin_deals',
'min_end_plan_non_fin_deals',
'min_start_fin_deals',
'min_start_non_fin_deals',
#'balance_amt_avg',
#'balance_amt_max',
#'balance_amt_min',
#'balance_amt_day_avg',
#'channel_code',
'city',
'city_type',
'index_city_code',
#'ogrn_days_end_month',
#'ogrn_days_end_quarter',
#'ogrn_month',
#'ogrn_year',
#'branch_code',
 #   'ft_registration_date', ////
'max_founderpres',
'min_founderpres',
#'ogrn_exist_months',
# 'okved',
#'segment',
#'sum_of_paym_2m',
#'sum_of_paym_6m',
#'sum_of_paym_1y',
#'sum_a_oper_1m',
#'cnt_a_oper_1m',
'sum_b_oper_1m',
#'cnt_b_oper_1m',
#'sum_c_oper_1m',
#'cnt_c_oper_1m',
#'sum_deb_d_oper_1m',
#'cnt_deb_d_oper_1m',
#'sum_cred_d_oper_1m',
#'cnt_cred_d_oper_1m',
#'sum_deb_e_oper_1m',
#'cnt_deb_e_oper_1m',
#'cnt_days_deb_e_oper_1m' //////////////
#'sum_cred_e_oper_1m',
#'cnt_cred_e_oper_1m',
#'cnt_days_cred_e_oper_1m', ////////////
#'sum_deb_f_oper_1m',
#'cnt_deb_f_oper_1m',
#'cnt_days_deb_f_oper_1m',
'sum_cred_f_oper_1m',
#'cnt_cred_f_oper_1m',
#'cnt_days_cred_f_oper_1m',
#'sum_deb_g_oper_1m',
#'cnt_deb_g_oper_1m',
#'cnt_days_deb_g_oper_1m',
#'sum_cred_g_oper_1m',
#'cnt_cred_g_oper_1m',
#'cnt_days_cred_g_oper_1m',
#'sum_deb_h_oper_1m',
#'cnt_deb_h_oper_1m',
#'cnt_days_deb_h_oper_1m',
#'sum_cred_h_oper_1m',
#'cnt_cred_h_oper_1m',
#'cnt_days_cred_h_oper_1m',
#'sum_a_oper_3m',
#'cnt_a_oper_3m',
'sum_b_oper_3m',
'cnt_b_oper_3m',
#'sum_c_oper_3m',
#'cnt_c_oper_3m',
#'sum_deb_d_oper_3m',
#'cnt_deb_d_oper_3m',
#'sum_cred_d_oper_3m',
#'cnt_cred_d_oper_3m',
#'sum_deb_e_oper_3m',
#'cnt_deb_e_oper_3m',
#'cnt_days_deb_e_oper_3m', ///////////
#'sum_cred_e_oper_3m',
#'cnt_cred_e_oper_3m',
#'cnt_days_cred_e_oper_3m', //////////
#'sum_deb_f_oper_3m',
#'cnt_deb_f_oper_3m',
#'cnt_days_deb_f_oper_3m',
#'sum_cred_f_oper_3m',
#'cnt_cred_f_oper_3m',
#'cnt_days_cred_f_oper_3m',
#'sum_deb_g_oper_3m',
#'cnt_deb_g_oper_3m',
#'cnt_days_deb_g_oper_3m',
#'sum_cred_g_oper_3m',
#'cnt_cred_g_oper_3m',
#'cnt_days_cred_g_oper_3m',
#'sum_deb_h_oper_3m',
#'cnt_deb_h_oper_3m',
#'cnt_days_deb_h_oper_3m'
#'sum_cred_h_oper_3m',
#'cnt_cred_h_oper_3m',
#'cnt_days_cred_h_oper_3m', ///////////
], axis=1, inplace=True)

In [ ]:
X = train_df.drop("total_target", axis=1)
y = train_df.total_target
x_train, x_val, y_train, y_val = train_test_split(X, y,
                                                  test_size=0.2,
                                                  random_state=42)

In [ ]:
test_df = pd.read_parquet('test.parquet', engine='fastparquet')

In [ ]:
# test_df.channel_code = pd.Categorical(test_df.channel_code)
# test_df['code1'] = test_df.channel_code.cat.codes
test_df.city = pd.Categorical(test_df.city)
test_df['code2'] = test_df.city.cat.codes
test_df.city_type = pd.Categorical(test_df.city_type)
test_df['code3'] = test_df.city_type.cat.codes
# # #test_df.index_city_code = pd.Categorical(test_df.index_city_code)
# # #test_df['code4'] = test_df.index_city_code.cat.codes
test_df.ogrn_month = pd.Categorical(test_df.ogrn_month)
test_df['code5'] = test_df.ogrn_month.cat.codes
# test_df.ogrn_year = pd.Categorical(test_df.ogrn_year)
# test_df['code6'] = test_df.ogrn_year.cat.codes
test_df.branch_code = pd.Categorical(test_df.branch_code)
test_df['code7'] = test_df.branch_code.cat.codes
test_df.okved = pd.Categorical(test_df.okved)
test_df['code8'] = test_df.okved.cat.codes
test_df.segment = pd.Categorical(test_df.segment)
test_df['code9'] = test_df.segment.cat.codes

In [ ]:
test_df["result_of_cor_1"] = test_df.cnt_cred_d_oper_3m * test_df.cnt_cred_d_oper_1m
test_df["result_of_cor_2"] = test_df.sum_cred_e_oper_1m * test_df.sum_deb_e_oper_1m
test_df["result_of_cor_3"] = test_df.cnt_days_deb_g_oper_3m * test_df.cnt_days_deb_g_oper_1m
test_df["result_of_cor_4"] = test_df.cnt_deb_h_oper_3m * test_df.cnt_deb_h_oper_1m
test_df["result_of_cor_5"] = test_df.cnt_cred_g_oper_3m * test_df.cnt_cred_g_oper_1m# - берем только 4 штуки т.к. результат лучше
test_df["result_of_cor_6"] = test_df.cnt_cred_f_oper_3m * test_df.cnt_cred_f_oper_1m
test_df["result_of_cor_7"] = test_df.ft_registration_date * test_df.ogrn_exist_months
test_df["result_of_cor_8"] = test_df.cnt_deb_f_oper_3m * test_df.cnt_cred_f_oper_1m
test_df["result_of_cor_10"] = test_df.cnt_cred_e_oper_3m * test_df.cnt_cred_e_oper_1m
test_df["result_of_cor_11"] = test_df.cnt_days_cred_h_oper_1m * test_df.cnt_days_cred_h_oper_3m
test_df["result_of_cor_12"] = test_df.cnt_cred_f_oper_1m * test_df.cnt_deb_f_oper_3m
test_df["result_of_cor_13"] = test_df.cnt_days_cred_f_oper_1m * test_df.cnt_deb_f_oper_1m
test_df["result_of_cor_14"] = test_df.cnt_cred_h_oper_1m * test_df.cnt_cred_h_oper_3m
test_df["result_of_cor_15"] = test_df.cnt_days_deb_f_oper_1m * test_df.cnt_cred_f_oper_3m

In [ ]:
test_df.info()

In [ ]:
test_df.head

In [ ]:
#test_df.hist(figsize=(40,40))

In [ ]:
#test_df['cnt_days_deb_h_oper_3m'].unique()

In [ ]:
#test_df['cnt_deb_h_oper_3m'].value_counts()

In [ ]:
#test_df['cnt_deb_h_oper_1m'].unique()

In [ ]:
#test_df['cnt_deb_h_oper_3m'].value_counts(normalize=True).plot.bar

In [ ]:
# обучение модели

In [ ]:
model = LGBMClassifier(verbosity=-1, random_state=100)
model.fit(x_train, y_train)

In [ ]:
y_pred = model.predict_proba(x_val)[:, 1]
roc_auc_score(y_val, y_pred)

In [ ]:
test_df[cat_cols] = test_df[cat_cols].astype("category")

In [ ]:
test_df.drop([
#'rko_start_months',
'max_end_fact_fin_deals',
'max_end_plan_non_fin_deals',
'max_start_fin_deals',
'max_start_non_fin_deals',
'min_end_fact_fin_deals',
'min_end_plan_non_fin_deals',
'min_start_fin_deals',
'min_start_non_fin_deals',
#'balance_amt_avg',
#'balance_amt_max',
#'balance_amt_min',
#'balance_amt_day_avg',
#'channel_code',
'city',
'city_type',
'index_city_code',
#'ogrn_days_end_month',
#'ogrn_days_end_quarter',
#'ogrn_month',
#'ogrn_year',
#'branch_code',
 #   'ft_registration_date', ////
'max_founderpres',
'min_founderpres',
#'ogrn_exist_months',
# 'okved',
#'segment',
#'sum_of_paym_2m',
#'sum_of_paym_6m',
#'sum_of_paym_1y',
#'sum_a_oper_1m',
#'cnt_a_oper_1m',
'sum_b_oper_1m',
#'cnt_b_oper_1m',
#'sum_c_oper_1m',
#'cnt_c_oper_1m',
#'sum_deb_d_oper_1m',
#'cnt_deb_d_oper_1m',
#'sum_cred_d_oper_1m',
#'cnt_cred_d_oper_1m',
#'sum_deb_e_oper_1m',
#'cnt_deb_e_oper_1m',
#'cnt_days_deb_e_oper_1m' //////////////
#'sum_cred_e_oper_1m',
#'cnt_cred_e_oper_1m',
#'cnt_days_cred_e_oper_1m', ////////////
#'sum_deb_f_oper_1m',
#'cnt_deb_f_oper_1m',
#'cnt_days_deb_f_oper_1m',
'sum_cred_f_oper_1m',
#'cnt_cred_f_oper_1m',
#'cnt_days_cred_f_oper_1m',
#'sum_deb_g_oper_1m',
#'cnt_deb_g_oper_1m',
#'cnt_days_deb_g_oper_1m',
#'sum_cred_g_oper_1m',
#'cnt_cred_g_oper_1m',
#'cnt_days_cred_g_oper_1m',
#'sum_deb_h_oper_1m',
#'cnt_deb_h_oper_1m',
#'cnt_days_deb_h_oper_1m',
#'sum_cred_h_oper_1m',
#'cnt_cred_h_oper_1m',
#'cnt_days_cred_h_oper_1m',
#'sum_a_oper_3m',
#'cnt_a_oper_3m',
'sum_b_oper_3m',
'cnt_b_oper_3m',
#'sum_c_oper_3m',
#'cnt_c_oper_3m',
#'sum_deb_d_oper_3m',
#'cnt_deb_d_oper_3m',
#'sum_cred_d_oper_3m',
#'cnt_cred_d_oper_3m',
#'sum_deb_e_oper_3m',
#'cnt_deb_e_oper_3m',
#'cnt_days_deb_e_oper_3m', ///////////
#'sum_cred_e_oper_3m',
#'cnt_cred_e_oper_3m',
#'cnt_days_cred_e_oper_3m', //////////
#'sum_deb_f_oper_3m',
#'cnt_deb_f_oper_3m',
#'cnt_days_deb_f_oper_3m',
#'sum_cred_f_oper_3m',
#'cnt_cred_f_oper_3m',
#'cnt_days_cred_f_oper_3m',
#'sum_deb_g_oper_3m',
#'cnt_deb_g_oper_3m',
#'cnt_days_deb_g_oper_3m',
#'sum_cred_g_oper_3m',
#'cnt_cred_g_oper_3m',
#'cnt_days_cred_g_oper_3m',
#'sum_deb_h_oper_3m',
#'cnt_deb_h_oper_3m',
#'cnt_days_deb_h_oper_3m'
#'sum_cred_h_oper_3m',
#'cnt_cred_h_oper_3m',
#'cnt_days_cred_h_oper_3m', ///////////
], axis=1, inplace=True)

In [ ]:
test_score = model.predict_proba(test_df.drop("id", axis=1))[:, 1]

In [ ]:
sample_submission_df = pd.read_csv("sample_submission.csv")

In [ ]:
sample_submission_df.head()

In [ ]:
sample_submission_df["score"] = test_score

In [ ]:
sample_submission_df.head()

In [ ]:
sample_submission_df.to_csv("my_submission.csv", index=False)